# Succinate production hands-on session

This notebook demonstrates how to load the yeast9 genome-scale metabolic model,
configure the extracellular medium, and explore environmental conditions that
maximize succinate secretion using flux balance analysis (FBA).


## 1. Environment preparation

The Python utilities shipped with the repository expect a `.env` file in the
project root and a working COBRApy installation. Create and activate a virtual
environment, install the dependencies, and ensure that the working directory of
this notebook is the repository root.

```bash
python -m venv .venv
source .venv/bin/activate
pip install cobra python-dotenv pandas
touch .env
```

Once the prerequisites are installed, the remaining cells can be executed in
sequence.


In [ ]:
from pathlib import Path

# Ensure that the repository root contains the marker file required by code.io
Path(".env").touch()
print('`.env` file ready for use.')


In [ ]:
import itertools
from typing import Dict, Iterable

import cobra
import pandas as pd
from cobra.flux_analysis import pfba

from code.io import read_yeast_model


## 2. Load the yeast9 model

The helper `read_yeast_model` locates the SBML file shipped with the repository
and returns a COBRApy `Model` instance.


In [ ]:
model = read_yeast_model()

print(f'Reactions: {len(model.reactions)}')
print(f'Metabolites: {len(model.metabolites)}')
print(f'Genes: {len(model.genes)}')
print(f'Default objective: {model.objective.expression}')


## 3. Configure a base medium

We reproduce the Kennedy synthetic complete medium used in the MATLAB utilities
(`complete_Y7`). All exchange reactions are first reset to prohibit uptake, then
selected nutrients are reopened with specific uptake bounds (negative flux
corresponds to import).


In [ ]:
# Exchange reactions that receive a constrained uptake of -0.5 mmol gDW^-1 h^-1
CONSTRAINED_UPTAKE: Iterable[str] = (
    'r_1604', 'r_1639', 'r_1873', 'r_1879', 'r_1880', 'r_1881', 'r_1671',
    'r_1883', 'r_1757', 'r_1891', 'r_1889', 'r_1810', 'r_1993', 'r_1893',
    'r_1897', 'r_1947', 'r_1899', 'r_1900', 'r_1902', 'r_1967', 'r_1903',
    'r_1548', 'r_1904', 'r_2028', 'r_2038', 'r_1906', 'r_2067', 'r_1911',
    'r_1912', 'r_1913', 'r_2090', 'r_1914', 'r_2106',
)

# Exchange reactions that remain unconstrained for uptake (-1000 lower bound)
UNCONSTRAINED_UPTAKE: Iterable[str] = (
    'r_1672',  # ammonium
    'r_1654',  # potassium
    'r_1992',  # oxygen
    'r_2005',  # phosphate
    'r_2060',  # sulfate
    'r_1861',  # iron
    'r_1832',  # proton
    'r_2100',  # water
    'r_4593',  # chloride
    'r_4595',  # manganese
    'r_4596',  # zinc
    'r_4597',  # magnesium
    'r_2049',  # sodium
    'r_4594',  # copper
    'r_4600',  # calcium
    'r_2020',  # sulfate/glutathione coupled import
)

GLUCOSE_EXCHANGE_ID = 'r_1714'
SUCCINATE_EXCHANGE_ID = 'r_2056'
BIOMASS_REACTION_ID = 'r_2111'

def apply_kennedy_medium(model: cobra.Model, glucose_limit: float = -20.0) -> cobra.Model:
    """Reset exchange bounds and apply the Kennedy synthetic complete medium."""
    for rxn in model.exchanges:
        rxn.lower_bound = 0.0
        rxn.upper_bound = 1000.0
    for rxn_id in CONSTRAINED_UPTAKE:
        model.reactions.get_by_id(rxn_id).lower_bound = -0.5
    model.reactions.get_by_id(GLUCOSE_EXCHANGE_ID).lower_bound = glucose_limit
    for rxn_id in UNCONSTRAINED_UPTAKE:
        model.reactions.get_by_id(rxn_id).lower_bound = -1000.0
    return model


In [ ]:
baseline_model = read_yeast_model()
apply_kennedy_medium(baseline_model)

succinate_exchange = baseline_model.reactions.get_by_id(SUCCINATE_EXCHANGE_ID)
baseline_model.objective = succinate_exchange
baseline_solution = baseline_model.optimize()

print(f'Baseline succinate flux: {baseline_solution.objective_value:.4f} mmol gDW^-1 h^-1')
print(f'Associated biomass flux: {baseline_solution.fluxes.get(BIOMASS_REACTION_ID, float("nan")):.4f}')
print(f'Optimization status: {baseline_solution.status}')


## 4. Explore environmental scenarios

We now parameterize a helper that reapplies the base medium, tweaks a set of
environmental constraints (glucose feed, oxygen availability, proton uptake as a
proxy for extracellular pH), and maximizes succinate secretion. The results are
tabulated for easy comparison.


In [ ]:
def evaluate_environment(
    model: cobra.Model,
    glucose_limit: float,
    oxygen_limit: float,
    proton_limit: float,
    ammonium_limit: float = -10.0,
    temperature_tag: str = '30C',
) -> Dict[str, float]:
    """Solve an FBA problem that maximizes succinate secretion."""
    with model:
        apply_kennedy_medium(model, glucose_limit=glucose_limit)
        model.reactions.get_by_id('r_1992').lower_bound = oxygen_limit
        model.reactions.get_by_id('r_1832').lower_bound = proton_limit
        model.reactions.get_by_id('r_1672').lower_bound = ammonium_limit

        # Optionally adjust non-growth-associated maintenance to mimic temperature shifts
        if temperature_tag == '37C':
            model.reactions.get_by_id('r_4041').lower_bound = 10.0
        elif temperature_tag == '20C':
            model.reactions.get_by_id('r_4041').lower_bound = 6.0

        model.objective = model.reactions.get_by_id(SUCCINATE_EXCHANGE_ID)
        solution = model.optimize()

        biomass_flux = solution.fluxes.get(BIOMASS_REACTION_ID, float('nan'))
        return {
            'status': solution.status,
            'succinate_flux': solution.objective_value,
            'biomass_flux': biomass_flux,
            'glucose_limit': glucose_limit,
            'oxygen_limit': oxygen_limit,
            'proton_limit': proton_limit,
            'ammonium_limit': ammonium_limit,
            'temperature_tag': temperature_tag,
        }


In [ ]:
analysis_model = read_yeast_model()
apply_kennedy_medium(analysis_model)

glucose_grid = (-10.0, -15.0, -20.0)
oxygen_grid = (0.0, -5.0, -10.0)
proton_grid = (-1.0, -5.0, -10.0)
temperature_grid = ('20C', '30C', '37C')

records = []
for glucose_limit, oxygen_limit, proton_limit, temperature_tag in itertools.product(
    glucose_grid, oxygen_grid, proton_grid, temperature_grid
):
    record = evaluate_environment(
        analysis_model,
        glucose_limit=glucose_limit,
        oxygen_limit=oxygen_limit,
        proton_limit=proton_limit,
        ammonium_limit=-10.0,
        temperature_tag=temperature_tag,
    )
    records.append(record)

results = pd.DataFrame(records)
results.sort_values('succinate_flux', ascending=False).head(10)


## 5. Inspect individual designs

Filter the table to retain feasible solutions (`status == 'optimal'`) and examine
the trade-off between succinate secretion and biomass growth to prioritize
conditions that remain physiologically realistic.


In [ ]:
optimal_results = results[results['status'] == 'optimal'].copy()
optimal_results = optimal_results.sort_values(['succinate_flux', 'biomass_flux'], ascending=[False, False])
optimal_results.head(10)


## 6. Extract flux distributions for promising scenarios

Use parsimonious FBA (pFBA) to compute flux distributions for specific scenarios.
This helps identify key pathways responsible for succinate overproduction under
the selected environmental settings.


In [ ]:
def sample_flux_distribution(row: pd.Series, model: cobra.Model) -> pd.Series:
    with model:
        apply_kennedy_medium(model, glucose_limit=row['glucose_limit'])
        model.reactions.get_by_id('r_1992').lower_bound = row['oxygen_limit']
        model.reactions.get_by_id('r_1832').lower_bound = row['proton_limit']
        model.reactions.get_by_id('r_1672').lower_bound = row['ammonium_limit']
        if row['temperature_tag'] == '37C':
            model.reactions.get_by_id('r_4041').lower_bound = 10.0
        elif row['temperature_tag'] == '20C':
            model.reactions.get_by_id('r_4041').lower_bound = 6.0
        model.objective = model.reactions.get_by_id(SUCCINATE_EXCHANGE_ID)
        pfba_solution = pfba(model)
    return pfba_solution.fluxes

top_design = optimal_results.iloc[0]
top_fluxes = sample_flux_distribution(top_design, analysis_model)
top_fluxes[top_fluxes.abs() > 1e-3].sort_values(ascending=False).head(20)


## 7. Next steps

* Constrain succinate secretion to a fixed minimum and re-maximize biomass to
  benchmark trade-offs between growth and production.
* Introduce additional feeds (e.g., glycerol or acetate) by adjusting the
  corresponding exchange lower bounds before solving the optimization.
* Export the most promising flux distributions and perform pathway enrichment or
  shadow price analysis to understand metabolic bottlenecks.
